In [ ]:
!pip install Augmentor

In [ ]:
import Augmentor
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from keras.layers import GlobalAveragePooling2D, Flatten, BatchNormalization, Dense, Dropout
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D

In [ ]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/images.zip', 'r') as zipObj:

   zipObj.extractall('IMAGES')

In [ ]:
p = Augmentor.Pipeline("/content/IMAGES/images",output_directory="/content/output")

Initialised with 210 image(s) found.
Output directory set to /content/output.

In [ ]:
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)

p.zoom(probability=0.3, min_factor=1.1, max_factor=1.6)

In [ ]:
p.sample(1000)

Processing <PIL.Image.Image image mode=RGB size=816x460 at 0x7FEF59E7FD90>: 100%|██████████| 1000/1000 [00:47<00:00, 21.21 Samples/s]


In [ ]:
import pathlib
data_dir = pathlib.Path('/content/output')
img_height=224
img_width=224

In [ ]:
import shutil
shutil.make_archive('/content/output', 'zip', root_dir='/content/drive/MyDrive/Dataset')

'/content/output.zip'

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=32
)


Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Found 2000 files belonging to 2 classes.
Using 400 files for validation.


In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [ ]:
normalization_layer = Rescaling(1./255)
train_ds_normalized = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds_normalized = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
num_classes=2

In [ ]:
def one_hot_encoding(x, y):
    return x, tf.one_hot(y, depth=num_classes)

train_ds_encoded = train_ds_normalized.map(one_hot_encoding)
val_ds_encoded = val_ds_normalized.map(one_hot_encoding)

In [ ]:
train_ds_encoded = train_ds_encoded.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds_encoded = val_ds_encoded.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
# Checkpoint to save best model per epoch

model_filepath = "/content/drive/My Drive/model-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath=model_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model_res50 = tf.keras.models.Sequential()
model_res50.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet'
))

model_res50.add(Dense(num_classes, activation='softmax'))


model_res50.layers[0].trainable = False

model_res50.summary()


model_res50.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])


history2 = model_res50.fit(
    train_ds_encoded,
    epochs=25,
    validation_data=val_ds_encoded,
    verbose=1
)

In [ ]:
model_res50.save('model_res50.h5') #Resnet50

In [ ]:
test_label = np.concatenate([y for x, y in val_ds_normalized], axis=0)
print(test_label.shape)

(400,)


In [ ]:
import matplotlib.pyplot as plt
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
loss = history2.history['loss']
val_loss = history2.history['val_loss']

In [ ]:
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
Y_pred = model_res50.predict_generator(val_ds_normalized, 1600)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm=confusion_matrix(test_label, y_pred)
print(cm)
print('Classification Report')
target_names = ['Pothole','SpeedBreaker']
print(classification_report(test_label, y_pred, target_names=target_names))

In [ ]:
import seaborn as sns
sns.set(font_scale=1.0)
ax=sns.heatmap(cm, annot=True, cmap='rocket', cbar=False, linewidths=3, linecolor='r', square=True, xticklabels=target_names,yticklabels=target_names,fmt='')
sns.set(font_scale = 2.0)
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nActual Values')
ax.set_ylabel('Predicted Values ')

-------------------------------** *** ****END OF** **RESNET50**-------------------------

------STARTING OF RESNET101------

In [ ]:
from tensorflow.keras.applications import ResNet101

In [ ]:
model_res101 = tf.keras.models.Sequential()
model_res101.add(ResNet101(
    include_top=False,
    pooling='avg',
    weights='imagenet'
))

In [ ]:
model_res101.add(Dense(num_classes, activation='softmax'))


model_res101.layers[0].trainable = False

model_res101.summary()

# Compile the model
model_res101.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 2048)              42658176  
                                                                 
 dense_1 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 42,662,274
Trainable params: 4,098
Non-trainable params: 42,658,176
_________________________________________________________________


In [ ]:
history3 = model_res101.fit(
    train_ds_encoded,
    epochs=25,
    validation_data=val_ds_encoded,
    verbose=1
)

Epoch 1/25
50/50 [==============================] - 24s 302ms/step - loss: 0.6857 - accuracy: 0.5581 - val_loss: 0.6574 - val_accuracy: 0.5750
Epoch 2/25
50/50 [==============================] - 11s 213ms/step - loss: 0.6407 - accuracy: 0.6694 - val_loss: 0.6120 - val_accuracy: 0.7700
Epoch 3/25
50/50 [==============================] - 11s 216ms/step - loss: 0.6023 - accuracy: 0.7531 - val_loss: 0.5811 - val_accuracy: 0.7775
Epoch 4/25
50/50 [==============================] - 11s 218ms/step - loss: 0.5704 - accuracy: 0.7950 - val_loss: 0.5548 - val_accuracy: 0.7975


KeyboardInterrupt: ignored

In [ ]:
model_res101.save('model_resnet101.h5')

In [ ]:
import matplotlib.pyplot as plt
acc = history3.history['accuracy']
val_acc = history3.history['val_accuracy']
loss = history3.history['loss']
val_loss = history3.history['val_loss']

In [ ]:
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
Y_pred = model_res101.predict_generator(val_ds_normalized, 1600)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm=confusion_matrix(test_label, y_pred)
print(cm)
print('Classification Report')
target_names = ['Pothole','SpeedBreaker']
print(classification_report(test_label, y_pred, target_names=target_names))

In [ ]:
import seaborn as sns
sns.set(font_scale=1.0)
ax=sns.heatmap(cm, annot=True, cmap='rocket', cbar=False, linewidths=3, linecolor='r', square=True, xticklabels=target_names,yticklabels=target_names,fmt='')
sns.set(font_scale = 2.0)
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nActual Values')
ax.set_ylabel('Predicted Values ')

-------END OF RESNET-101-----------

--------STARTING RESNET-152-------

In [ ]:
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
resnet152_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in resnet152_model.layers:
    layer.trainable = False


x = resnet152_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x= Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
model_res152 = Model(inputs=resnet152_model.input, outputs=predictions)

In [ ]:
model_res152.compile(optimizer=Adam(learning_rate=0.0001),
                     loss=CategoricalCrossentropy(),
                     metrics=['accuracy'])

In [ ]:
history4 = model_res152.fit(
    train_ds_encoded,
    epochs=25,
    validation_data=val_ds_encoded,
    verbose=1
)

In [ ]:
acc = history4.history['accuracy']
val_acc = history4.history['val_accuracy']
loss = history4.history['loss']
val_loss = history4.history['val_loss']

In [ ]:
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model_res152.save("model_res152.h5")

In [ ]:
train_label = np.concatenate([y for x, y in train_ds_encoded], axis=0)
print(train_label.shape)

In [ ]:
test_label = np.concatenate([y for x, y in val_ds_normalized], axis=0)
print(test_label.shape)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
Y_pred = model_res152.predict_generator(val_ds_normalized, 2000)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm=confusion_matrix(test_label, y_pred)
print(cm)
print('Classification Report')
target_names = ['Pothole','SpeedBreaker']
print(classification_report(test_label, y_pred, target_names=target_names))

In [ ]:
import seaborn as sns
sns.set(font_scale=1.0)
ax=sns.heatmap(cm, annot=True, cmap='rocket', cbar=False, linewidths=3, linecolor='r', square=True, xticklabels=target_names,yticklabels=target_names,fmt='')
sns.set(font_scale = 2.0)
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nActual Values')
ax.set_ylabel('Predicted Values ')

ENSEMBLED MODEL

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
model_1 = load_model('/content/model_res50.h5')
model_1 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='name_of_model_1')
model_2 = load_model('/content/model_resnet101.h5')
model_2 = Model(inputs=model_2.inputs,
                outputs=model_2.outputs,
                name='name_of_model_2')
models = [model_1, model_2]
model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

ensemble_model.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
history6=ensemble_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=25)

In [ ]:
acc = history6.history['accuracy']
val_acc = history6.history['val_accuracy']
loss = history6.history['loss']
val_loss = history6.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
train_label = np.concatenate([y for x, y in train_ds], axis=0)
print(train_label.shape)

In [ ]:
test_label = np.concatenate([y for x, y in val_ds], axis=0)
print(test_label.shape)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
Y_pred = ensemble_model.predict_generator(val_ds, 1600)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm=confusion_matrix(test_label, y_pred)
print(cm)
print('Classification Report')
target_names = ['Pothole','SpeedBreaker']
print(classification_report(test_label, y_pred, target_names=target_names))

In [ ]:
import seaborn as sns
sns.set(font_scale=1.0)
ax=sns.heatmap(cm, annot=True, cmap='summer', cbar=False, linewidths=3, linecolor='r', square=True, xticklabels=target_names,yticklabels=target_names,fmt='')
sns.set(font_scale = 2.0)
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nActual Values')
ax.set_ylabel('Predicted Values ')